In [1]:
# figure 1 - 2D plots
path <- '/Users/dongmeichen/Documents/beetle/data'
df.fire <- read.csv(sprintf('%s/plot_2D_firedata.csv', path))
df.forest <- read.csv(sprintf('%s/plot_2D_forestdata.csv', path))
df <- cbind(df.fire, df.forest)
head(df)

SprsFires,SprsFires.y,PctSprs,PctSprs.y,SprsAcre,SprsAcre.y,mStdAge,mStdAge.y,density,density.y,vcc,vcc.y,GAP1,GAP1.y,GAP2,GAP2.y,GAP3,GAP3.y
1.000000,103.4801,0.02564103,144.9659,0.05000,107.4667,12.16451,443.8910,1339.039,216.1387,1.000000,450.6334,0.00000000,541.4758,0.00000000,541.4758,0.00000000,491.7646
1.424242,104.3126,0.03548304,144.0030,38.88346,107.3240,14.27779,447.0451,1907.403,220.4782,1.040404,453.6270,0.01010101,539.9230,0.01010101,542.2181,0.01010101,492.2432
1.848485,105.1519,0.04532505,143.0465,77.71692,107.1814,16.39107,450.1817,2475.768,224.8805,1.080808,456.5253,0.02020202,538.3747,0.02020202,542.9613,0.02020202,492.7222
2.272727,105.9978,0.05516706,142.0963,116.55038,107.0391,18.50435,453.3002,3044.133,229.3459,1.121212,459.3347,0.03030303,536.8308,0.03030303,543.7056,0.03030303,493.2017
2.696970,106.8506,0.06500907,141.1525,155.38384,106.8969,20.61763,456.3997,3612.498,233.8748,1.161616,462.0610,0.04040404,535.2913,0.04040404,544.4509,0.04040404,493.6817
3.121212,107.7103,0.07485107,140.2149,194.21730,106.7549,22.73090,459.4797,4180.863,238.4673,1.202020,464.7094,0.05050505,533.7562,0.05050505,545.1972,0.05050505,494.1621
